In [1]:
import os
import math
import config
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm

from utils import (
    create_img_grid,
    SpatialProjection
)

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [2]:
data_dir = "../../data/dataset/raw/"
subjects = os.listdir(data_dir)

augmentation_levels = [0, 7, 9, 11, 13]
sp_augment = [
    SpatialProjection(
    img_dir="../../data/dataset/images/",
    img_len=math.floor(config.IMG_LEN / 3),
    polyfit_degree=degree
)
    for degree in augmentation_levels ]



In [3]:
import random
random.randint(1, 100)

17

In [4]:
train_images = []
train_labels = []
test_images = []
test_labels = []

for subject in tqdm(subjects):
    for gesture in config.GESTURES:
        gesture_dir = os.path.join(data_dir, subject, gesture)
        recordings = os.listdir(gesture_dir)
        for recording in recordings:
            _images = []
            file_path = os.path.join(gesture_dir, recording)

            data = pd.read_csv(file_path)
            data.drop(columns=["time"], inplace=True)
            data.drop(0, inplace=True)  # Remove first All-0 row

            if data.shape[0] == 0:
                continue

            # Flag for determining Trainning and Testing Samples
            for_training = random.randint(1, 100) <= 80

            for sp in sp_augment:
                for landmark in config.PROJECTION_LANDMARKS:
                    _images.extend(
                        sp.get_projection_images(
                            data=data.filter(regex=landmark),
                            subject=subject,
                            gesture=gesture
                        )
                    )

                img = create_img_grid(_images, config.IMG_LEN)

                if for_training:
                    train_images.append(img)
                    train_labels.append(config.GESTURES.index(gesture))
                else:
                    test_images.append(img)
                    test_labels.append(config.GESTURES.index(gesture))
                    break

    #         plt.imshow(img)
    #         plt.savefig("../assets/projection_demo.pdf")
    #         plt.show()

    #         break
    #     break
    # break


100%|██████████| 7/7 [07:37<00:00, 65.31s/it]


In [5]:
# X = np.array(images, dtype="uint8")
# X = np.expand_dims(X[:, :, :, 0], axis=-1)
# # X = np.repeat(np.expand_dims(X, axis=-1), 3, axis=-1)
# y = np.array(labels, dtype="uint8")

X_train = np.array(train_images, dtype="uint8")
X_test = np.array(test_images, dtype="uint8")
y_train = np.array(train_labels, dtype="uint8")
y_test = np.array(test_labels, dtype="uint8")

X_train = np.expand_dims(X_train[:, :, :, 0], axis=-1)
X_test = np.expand_dims(X_test[:, :, :, 0], axis=-1)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4650, 224, 224, 1)
(229, 224, 224, 1)
(4650,)
(229,)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.33, random_state=42
# )


In [ ]:
# preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
# base_model = tf.keras.applications.MobileNetV2(
#     input_shape=(config.IMG_LEN, config.IMG_LEN, 3),
#     include_top=False,
#     weights="imagenet"
# )
# base_model.trainable = True
# global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
# prediction_layer = tf.keras.layers.Dense(len(config.GESTURES))

# inputs = tf.keras.Input(shape=(config.IMG_LEN, config.IMG_LEN, 3))
# x = preprocess_input(inputs)
# x = base_model(x, training=True)
# x = global_average_layer(x)
# x = tf.keras.layers.Dropout(0.6)(x)
# outputs = prediction_layer(x)
# model = tf.keras.Model(inputs, outputs)


In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(
        1/255.0,
        input_shape=(config.IMG_LEN, config.IMG_LEN, config.N_CHANNELS)
    ),
    tf.keras.layers.Conv2D(16, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(16, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(config.GESTURES))
])


In [ ]:
# import config
# from model import ConvMixer

# model = ConvMixer(
#     img_size=config.IMG_LEN,
#     in_channels=1,
#     n_classes=len(config.GESTURES),
#     n_filters=64,
#     depth=3,
#     kernel_size=5,
#     patch_size=2
# ).model()



In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [8]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=30,
        restore_best_weights=True
    )
]

history = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=700,
    verbose=1,
    callbacks=callbacks
)


Epoch 1/700
146/146 [==============================] - 42s 284ms/step - loss: 2.5781 - accuracy: 0.1176 - val_loss: 2.2342 - val_accuracy: 0.2271
Epoch 2/700
146/146 [==============================] - 42s 286ms/step - loss: 1.5832 - accuracy: 0.5022 - val_loss: 0.9310 - val_accuracy: 0.7380
Epoch 3/700
146/146 [==============================] - 43s 293ms/step - loss: 0.9278 - accuracy: 0.6994 - val_loss: 0.6930 - val_accuracy: 0.7991
Epoch 4/700
146/146 [==============================] - 44s 299ms/step - loss: 0.6808 - accuracy: 0.7806 - val_loss: 0.5120 - val_accuracy: 0.8253
Epoch 5/700
146/146 [==============================] - 49s 335ms/step - loss: 0.5515 - accuracy: 0.8174 - val_loss: 0.4143 - val_accuracy: 0.8734
Epoch 6/700
146/146 [==============================] - 46s 313ms/step - loss: 0.4535 - accuracy: 0.8460 - val_loss: 0.3392 - val_accuracy: 0.9039
Epoch 7/700
146/146 [==============================] - 47s 319ms/step - loss: 0.4103 - accuracy: 0.8634 - val_loss: 0.2728 -

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({
    # "text.usetex": True,
    "font.family": "serif",
    # "font.serif": ["Computer Modern Roman"],
    "font.size": 22,
    "text.color": "#212121",
    "axes.edgecolor": "#212121",
    "xtick.color": "#212121",
    "ytick.color": "#212121",
    "axes.labelcolor": "#212121",
    'legend.frameon': False,
})

fig = plt.figure(figsize=(8, 6))
ax = fig.gca()
ax.plot(history.history["loss"], "-", color="#212121", label="Train Loss")
ax.plot(history.history["val_loss"], "--",
        color="#212121", label="Validation Loss")
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.title("Learning Curves")
plt.legend()
plt.tight_layout()
plt.savefig("../assets/lc.png")
plt.show()


In [ ]:
tf.keras.backend.clear_session()
